In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchinfo


In [3]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data_as_np):
        pass
    def __getitem__(self, index):
        pass
    def __len__(self):
        pass

In [2]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
    def forward(x):
        pass